In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from envs.mimic import MIMICEnv
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

from dddqn import DDDQN_Agent


In [2]:
env = gym.make('mimic-v0')


def eval_policy(model, env, verbose=True):

    returns, lengths = evaluate_policy(model, env, n_eval_episodes=20, return_episode_rewards=True)

    if verbose:
        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.set_title('return / episode')
        ax1.plot(returns)
        ax2.set_title('episode length')
        ax2.plot(lengths)

        plt.show()

    return np.mean(returns), np.mean(lengths)

In [3]:
### Proximal Policy Optimization ###

ppo_model = PPO('MlpPolicy', env, verbose=1).learn(total_timesteps=1000)
#ppo_model.save('models/mimic_ppo')

#ppo_model = PPO.load('models/mimic_ppo')

ppo_return, ppo_len = eval_policy(ppo_model, env)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/jakeweiss/miniconda3/envs/py39/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


ValueError: could not broadcast input array from shape (24,) into shape (20,)

In [ ]:
### Deep Q Networks ###

dqn_model = DQN('MlpPolicy', env, verbose=1).learn(total_timesteps=1000)
#dqn_model.save('models/mimic_dqn')

#dqn_model = DQN.load('models/mimic_dqn')

dqn_return, dqn_len = eval_policy(dqn_model, env)

In [ ]:
### Dueling Double Deep Q Networks)

dddqn_agent = DDDQN_Agent(env.observation_space.n, env.observation_space.n)